In [1]:
import math
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import rebound as rb

import reboundx as rx

In [2]:
from importlib import reload

In [3]:
reload (rb)

<module 'rebound' from '/opt/python/3.6.4/lib/python3.6/site-packages/rebound/__init__.py'>

In [4]:
reload(rx)

<module 'reboundx' from '/opt/python/3.6.4/lib/python3.6/site-packages/reboundx/__init__.py'>

In [5]:
rx.__version__

'2.18.1'

In [ ]:
def getsim():
    sim = rb.Simulation()
    sim.units = ('yr', 'AU', 'Msun')
    sim.add(m=1, r=0.1, hash="Binary 1")
    sim.add(m=1, r=0.1, a=1, e= 0.1, hash="Binary 2", f=np.random.rand()*2*np.pi)
    sim.move_to_com()
    rebx = rx.Extras(sim)
    tides = rebx.load_force("tides_constant_time_lag")
    rebx.add_force(tides)
    
    return sim, rebx, tides

In [ ]:
sim, rebx, tides = getsim()
ps = sim.particles
ps[0].params["tctl_k1"] = 0.5
# tau is zero for now as it's not specified

In [ ]:
help(rx.Extras)

In [ ]:
tmax = 2.5e4
Nout=1000

a, e = np.zeros(Nout), np.zeros(Nout)
times = np.linspace(0,tmax,Nout)
# to plot physical radius of the Sun
R0 = 0*times + ps[0].r

q = (ps[1].m/ps[0].m)
T = ps[0].r**3/sim.G/ps[0].m/ps[0].params["tctl_tau"]
apred = ps[0].r*((ps[1].a/ps[0].r)**8 - 48.*ps[0].params["tctl_k1"]*q*(1+q)*times/T)**(1./8.)

In [ ]:
%%time
for i, time in enumerate(times):
    sim.integrate(time)
    a[i] = ps[1].a
    e[i] = ps[1].e

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(times, a, label='$a_{\oplus}$')
ax.plot(times, R0, label='$R_{\odot}$')
ax.plot(times, apred, '--', label='$a_{\oplus}$ predicted')
ax.set_xlabel('$t$ (yr)', fontsize=24)
ax.set_ylabel('(AU)', fontsize=24)
ax.legend(fontsize=24, loc='best')